In [ ]:
import os
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, simpledialog, Listbox, Scrollbar
from PIL import Image, ImageTk  # For icons
import visualization
import subprocess  # Added for opening files (Line 9)
import platform 

In [ ]:
class DirectoryManager:
    def __init__(self, root):
        self.root = root
        self.root.title("🗂 Directory Manager")
        self.root.geometry("600x500")
        self.root.configure(bg="#f0f0f0")


        # 🔹 Add Keyboard Shortcuts 
        self.root.bind("<Control-n>", lambda e: self.create_file())       # Ctrl + N for new file
        self.root.bind("<Control-Shift-N>", lambda e: self.create_folder())  # Ctrl + Shift + N for folder
        self.root.bind("<Delete>", lambda e: self.delete_selected())      # Delete key
        self.root.bind("<Control-r>", lambda e: self.rename_selected())   # Ctrl + R for rename
        self.root.bind("<Control-q>", lambda e: self.exit_app())          # Ctrl + Q for exit


        # Directory Selection
        self.selected_dir = tk.StringVar(value=os.getcwd())

        top_frame = ttk.Frame(root, padding=10)
        top_frame.pack(fill="x")

        ttk.Button(top_frame, text="📂 Browse", command=self.browse_directory).pack(side="left")
        ttk.Label(top_frame, textvariable=self.selected_dir, wraplength=450).pack(side="left", padx=10)

        # File List Frame
        list_frame = ttk.Frame(root, padding=10)
        list_frame.pack(fill="both", expand=True)

        self.listbox = Listbox(list_frame, width=60, height=15, font=("Arial", 12))
        self.listbox.pack(side="left", fill="both", expand=True)

        scrollbar = Scrollbar(list_frame, command=self.listbox.yview)
        scrollbar.pack(side="right", fill="y")
        self.listbox.config(yscrollcommand=scrollbar.set)
        
        # Buttons Frame (Fixed - Does NOT scroll)
        btn_frame = ttk.Frame(root, padding=10)
        btn_frame.pack(fill="x")

        # Main Action Buttons (Fixed)
        
        ttk.Button(btn_frame, text="📂 Open", command=self.open_selected).pack(side="left", padx=5)
        ttk.Button(btn_frame, text="📄 Create File", command=self.create_file).pack(side="left", padx=5)
        ttk.Button(btn_frame, text="📁 Create Folder", command=self.create_folder).pack(side="left", padx=5)
        ttk.Button(btn_frame, text="✏️ Rename", command=self.rename_selected).pack(side="left", padx=5)
        ttk.Button(btn_frame, text="🗑 Delete", command=self.delete_selected).pack(side="left", padx=5)
        ttk.Button(root, text="🌙 Toggle Dark Mode", command=self.toggle_theme).pack(pady=5)  #Added Toggle Dark Mode Button


        # Visualization Buttons Frame with Horizontal Scrolling
        vis_container = ttk.Frame(root, padding=10)
        vis_container.pack(fill="x")

        # Create a Canvas for scrolling
        canvas = tk.Canvas(vis_container, height=40)
        canvas.pack(side="top", fill="x", expand=True)

        # Add Horizontal Scrollbar
        h_scrollbar = ttk.Scrollbar(vis_container, orient="horizontal", command=canvas.xview)
        h_scrollbar.pack(side="bottom", fill="x")

        # Configure Canvas for Horizontal Scrolling
        canvas.configure(xscrollcommand=h_scrollbar.set)
        canvas.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

        # Frame inside Canvas to hold Visualization Buttons
        vis_frame = ttk.Frame(canvas)
        canvas.create_window((0, 0), window=vis_frame, anchor="nw")

        # Add Visualization Buttons inside Scrollable Frame
        vis_buttons = [
            ("📊 File Type Distribution", lambda: visualization.DirectoryVisualizer.visualize_file_types_static(self.selected_dir.get())),
            ("📈 Largest Folders", lambda: visualization.DirectoryVisualizer.visualize_folder_sizes_static(self.selected_dir.get())),
            ("📂 Largest Files", lambda: visualization.DirectoryVisualizer.visualize_largest_files(self.selected_dir.get())),
            ("📅 File Age Analysis", lambda: visualization.DirectoryVisualizer.visualize_file_ages_static(self.selected_dir.get())),
            ("🔍 Find Duplicates", lambda: visualization.DirectoryVisualizer.detect_duplicate_files(self.selected_dir.get()))
        ]

        for text, command in vis_buttons:
            ttk.Button(vis_frame, text=text, command=command).pack(side="left", padx=5, pady=2)

        # Ensure scrolling works with mouse wheel (horizontal scroll)
        vis_frame.bind("<Enter>", lambda e: root.bind_all("<Shift-MouseWheel>", lambda event: canvas.xview_scroll(-1 * (event.delta // 120), "units")))
        vis_frame.bind("<Leave>", lambda e: root.unbind_all("<Shift-MouseWheel>"))
        
        # Exit Button
        ttk.Button(root, text="🚪 Exit", command=self.exit_app, style="Exit.TButton").pack(pady=10)

    def browse_directory(self):
        dir_path = filedialog.askdirectory()
        if dir_path:
            self.selected_dir.set(dir_path)
            self.refresh_list()

     def refresh_list(self):
        self.listbox.delete(0, tk.END)
        path = self.selected_dir.get()
        try:
            for item in os.listdir(path):
                item_path = os.path.join(path, item)
                size = os.path.getsize(item_path) if os.path.isfile(item_path) else 0  
                item_type = "📄" if os.path.isfile(item_path) else "📁"  
                self.listbox.insert(tk.END, f"{item_type} {item} ({size} bytes)") 
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def create_file(self):
        file_name = simpledialog.askstring("Create File", "Enter file name:")
        if file_name:
            file_path = os.path.join(self.selected_dir.get(), file_name)
            if os.path.exists(file_path):  #checks if there are files with the same name
                messagebox.showwarning("Warning", f"'{file_name}' already exists!")  
                return
            try:
                with open(file_path, "w") as f:
                    f.write("")  # Create an empty file
                messagebox.showinfo("Success", f"File '{file_name}' created successfully!")
            except Exception as e:
                messagebox.showerror("Error", str(e))

    def create_folder(self):
        folder_name = simpledialog.askstring("Create Folder", "Enter folder name:")
        if folder_name:
            folder_path = os.path.join(self.selected_dir.get(), folder_name)
            if os.path.exists(folder_path):  #checks if folder with the same name exists
                messagebox.showwarning("Warning", f"'{folder_name}' already exists!")  
                return
            try:
                os.makedirs(folder_path, exist_ok=True)
                messagebox.showinfo("Success", f"Folder '{folder_name}' created successfully!")
            except Exception as e:
                messagebox.showerror("Error", str(e))
                
    def rename_selected(self):
        selected = self.listbox.curselection()
        if not selected:
            messagebox.showwarning("Warning", "No file/folder selected!")
            return
        
        old_name = self.listbox.get(selected)
        old_path = os.path.join(self.selected_dir.get(), old_name)
        new_name = simpledialog.askstring("Rename", f"Enter new name for '{old_name}':")

        if new_name:
            new_path = os.path.join(self.selected_dir.get(), new_name)
            try:
                os.rename(old_path, new_path)
                self.refresh_list()
                messagebox.showinfo("Success", f"Renamed '{old_name}' to '{new_name}'!")
            except Exception as e:
                messagebox.showerror("Error", str(e))

    def delete_selected(self):
        selected = self.listbox.curselection()
        if not selected:
            messagebox.showwarning("Warning", "No file/folder selected!")
            return
        
        item_name = self.listbox.get(selected)
        item_path = os.path.join(self.selected_dir.get(), item_name)
        confirm = messagebox.askyesno("Confirm Delete", "Are you sure you want to delete this?")
        
        if confirm:
            try:
                if os.path.isdir(item_path):
                    os.rmdir(item_path)  # Deletes empty folder
                else:
                    os.remove(item_path)  # Deletes file
                self.refresh_list()
            except Exception as e:
                messagebox.showerror("Error", str(e))



    def open_selected(self):  # (🔹 Line 95)
       
        selected = self.listbox.curselection()
        if not selected:
            messagebox.showwarning("Warning", "No file/folder selected!")
            return

        item_name = self.listbox.get(selected)
        item_path = os.path.join(self.selected_dir.get(), item_name)

        try:
            if platform.system() == "Windows":
                os.startfile(item_path)
            elif platform.system() == "Darwin":  # macOS
                subprocess.call(["open", item_path])
            else:  # Linux
                subprocess.call(["xdg-open", item_path])
        except Exception as e:
            messagebox.showerror("Error", str(e))



    def toggle_theme(self):  # Added Dark Mode Toggle Feature
       
        dark_mode = self.root.option_get("dark_mode", "0") == "1"
        
        if dark_mode:
            self.root.tk_setPalette(background="#f0f0f0", foreground="black")
            self.listbox.config(bg="white", fg="black")
        else:
            self.root.tk_setPalette(background="#2e2e2e", foreground="white")
            self.listbox.config(bg="#333", fg="white")
        
        self.root.option_add("*TButton*foreground", "white" if not dark_mode else "black")
        self.root.option_add("*TButton*background", "#555" if not dark_mode else "#ddd")
        self.root.option_add("*dark_mode", "1" if not dark_mode else "0")




    def exit_app(self):
        confirm_exit = messagebox.askyesno("Exit", "Are you sure you want to exit?")
        if confirm_exit:
            self.root.update_idletasks()
            self.root.quit()

In [7]:
# Run the Tkinter application
if __name__ == "__main__":
    root = tk.Tk()
    app = DirectoryManager(root)
    root.mainloop()